In [1]:
from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
from keras.engine.topology import Layer
from keras import backend as K
K.set_image_data_format('channels_first')

import cv2
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
from fr_utils import *
from inception_blocks_v2 import *
import sklearn
import sys
import gc
import json
import shutil

%matplotlib inline
%load_ext autoreload
%autoreload 2

np.set_printoptions(threshold=np.nan)

Using TensorFlow backend.


In [ ]:
import pandas as pd

In [ ]:
import cv2

In [2]:
def triplet_loss(y_true, y_pred, alpha = 0.2):
    """
    Implementation of the triplet loss as defined by formula (3)
    
    Arguments:
    y_true -- true labels, required when you define a loss in Keras, you don't need it in this function.
    y_pred -- python list containing three objects:
            anchor -- the encodings for the anchor images, of shape (None, 128)
            positive -- the encodings for the positive images, of shape (None, 128)
            negative -- the encodings for the negative images, of shape (None, 128)
    
    Returns:
    loss -- real number, value of the loss
    """
    
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]

    # Step 1: Compute the (encoding) distance between the anchor and the positive, you will need to sum over axis=-1
    pos_dist = tf.reduce_sum(tf.square(anchor-positive),axis=-1)
    # Step 2: Compute the (encoding) distance between the anchor and the negative, you will need to sum over axis=-1
    neg_dist = tf.reduce_sum(tf.square(anchor-negative),axis=-1)
    # Step 3: subtract the two previous distances and add alpha.
    basic_loss = tf.add(tf.subtract(pos_dist,neg_dist),alpha)
    # Step 4: Take the maximum of basic_loss and 0.0. Sum over the training examples.
    loss = tf.reduce_sum(tf.maximum(basic_loss,0))

    
    return loss

In [3]:
#加载模型
FRmodel = faceRecoModel(input_shape=(3, 96, 96))
#print("Total Params:", FRmodel.count_params())
FRmodel.compile(optimizer = 'adam', loss = triplet_loss, metrics = ['accuracy'])
load_weights_from_FaceNet(FRmodel)

In [ ]:
database = {}
database["danielle"] = img_to_encoding("images/danielle.png", FRmodel)
database["younes"] = img_to_encoding("images/younes.jpg", FRmodel)
#database["tian"] = img_to_encoding("images/tian.jpg", FRmodel)
#database["andrew"] = img_to_encoding("images/andrew.jpg", FRmodel)
#database["kian"] = img_to_encoding("images/kian.jpg", FRmodel)
#database["dan"] = img_to_encoding("images/dan.jpg", FRmodel)
#database["sebastiano"] = img_to_encoding("images/sebastiano.jpg", FRmodel)
database["bertrand"] = img_to_encoding("images/bertrand.jpg", FRmodel)
database["kevin"] = img_to_encoding("images/kevin.jpg", FRmodel)
database["felix"] = img_to_encoding("images/felix.jpg", FRmodel)
database["benoit"] = img_to_encoding("images/benoit.jpg", FRmodel)
database["arnaud"] = img_to_encoding("images/arnaud.jpg", FRmodel)
database['lisa'] = img_to_encoding('images/lisa.jpg',FRmodel)
database['xing'] = img_to_encoding('images/xingtest2.jpg',FRmodel)
database['xing1'] = img_to_encoding('images/xing1.jpg',FRmodel)
database['xing2'] = img_to_encoding('images/xing2.jpg',FRmodel)
#database['xing3'] = img_to_encoding('images/xing3.jpg',FRmodel)

# function

In [4]:
IMAGE_SIZE = 96
def resize_image(image, height = IMAGE_SIZE, width = IMAGE_SIZE):
    top, bottom, left, right = (0, 0, 0, 0)
    
    #获取图像尺寸
    h, w, _ = image.shape
    
    #对于长宽不相等的图片，找到最长的一边
    longest_edge = max(h, w)    
    
    #计算短边需要增加多上像素宽度使其与长边等长
    if h < longest_edge:
        dh = longest_edge - h
        top = dh // 2
        bottom = dh - top
    elif w < longest_edge:
        dw = longest_edge - w
        left = dw // 2
        right = dw - left
    else:
        pass 
    
    #RGB颜色
    BLACK = [0, 0, 0]
    
    #给图像增加边界，是图片长、宽等长，cv2.BORDER_CONSTANT指定边界颜色由value指定
    constant = cv2.copyMakeBorder(image, top , bottom, left, right, cv2.BORDER_CONSTANT, value = BLACK)
    
    #调整图像大小并返回
    return cv2.resize(constant, (height, width))

def who_is_it(image, database, model):
    """
    Implements face recognition for the happy house by finding who is the person on the image_path image.
    
    Arguments:
    image_path -- path to an image
    database -- database containing image encodings along with the name of the person on the image
    model -- your Inception model instance in Keras
    
    Returns:
    min_dist -- the minimum distance between image_path encoding and the encodings from the database
    identity -- string, the name prediction for the person on image_path
    """
    
    
    ## Step 1: Compute the target "encoding" for the image. Use img_to_encoding() see example above
    encoding = image_to_encoding(image,model)
    
    ## Step 2: Find the closest encoding ##
    
    # Initialize "min_dist" to a large value
    min_dist = 100
    
    # Loop over the database dictionary's names and encodings.
    for (name, db_enc) in database.items():
        
        # Compute L2 distance between the target "encoding" and the current "emb" from the database
        dist = np.linalg.norm(encoding-db_enc)

        # If this distance is less than the min_dist, then set min_dist to dist, and identity to name
        if dist<min_dist:
            min_dist = dist
            identity = name


    '''
    if min_dist > 0.7:
        print("Not in the database.")
    else:
        print ("it's " + str(identity) + ", the distance is " + str(min_dist))
    '''    
    return min_dist, identity

def findNearestClassForImage(face_descriptor, faceLabel):
    temp =  face_descriptor - data
    e = np.linalg.norm(temp,axis=1,keepdims=True)
    min_distance = e.min() 
    #print('distance: ', min_distance)
    if min_distance > threshold:
        return 'other'
    index = np.argmin(e)
    return faceLabel[index]

# image emcodding

In [ ]:
imagePath = 'imgs/'                                                                           #图像的目录
data = np.zeros((1,128))                                                                            #定义一个128维的空向量data
label = []                                                                                          #定义空的list存放人脸的标签

for file in os.listdir(imagePath):                                                                  #开始一张一张索引目录中的图像
    if '.jpg' in file or '.png' in file:
        fileName = file
        labelName = file.split('_')[0]                                                              #获取标签名
        print('current image: ', file)
        print('current label: ', labelName)
        
        img = cv2.imread(imagePath + file)                                                          #使用opencv读取图像数据
        if img.shape[0]*img.shape[1] > 500000:                                                      #如果图太大的话需要压缩，这里像素的阈值可以自己设置
            img = cv2.resize(img, (0,0), fx=0.5, fy=0.5)
            
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)#将图片转化成灰度

        face_cascade = cv2.CascadeClassifier("D:\\WM\\AI learning\\Ng - Deep Learning\\course4\\face recognistion\\haarcascade_frontalface_alt2.xml")
        #face_cascade.load('/haarcascade_frontalface_alt2.xml')#一定要告诉编译器文件所在的具体位置
        #'''此文件是opencv的haar人脸特征分类器'''
        faces = face_cascade.detectMultiScale(gray, 1.2,  minNeighbors = 3, minSize = (32, 32))
        #print(faces)
        #image_set=[]
        for (x,y,w,h) in faces: 
            
        #dets = detector(img, 1)                                                                     #使用检测算子检测人脸，返回的是所有的检测到的人脸区域
        #for k, d in enumerate(dets):
            image = img[y - 10: y + h + 10, x - 10: x + w + 10]
            image = resize_image(image)
            embeddings = image_to_encoding(image, FRmodel)                                                                   #获取landmark
            #faceArray = np.array(face_descriptor).reshape((1, 128))                                 #转换成numpy中的数据结构
            data = np.concatenate((data, embeddings))                                                #拼接到事先准备好的data当中去
            label.append(labelName)                                                                 #保存标签
            cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)       #显示人脸区域
            
        cv2.namedWindow('img',1)#O表示显示窗口可以随意手动调节，1
        cv2.imshow('img',img)
        cv2.waitKey()
        cv2.destroyAllWindows()


data = data[1:, :]                                                                                  #因为data的第一行是空的128维向量，所以实际存储的时候从第二行开始
np.savetxt('faceData.txt', data, fmt='%f')                                                          #保存人脸特征向量合成的矩阵到本地

labelFile=open('label.txt','w')                                      
json.dump(label, labelFile)                                                                         #使用json保存list到本地
labelFile.close()

cv2.destroyAllWindows()                                                                             #关闭所有的窗口

# camera capture photos

In [5]:
#-*- coding: utf-8 -*-
# save the photos

if __name__ == '__main__':

    # open data and label file
    labelFile=open('label.txt','r')
    label = json.load(labelFile)                                                   #载入本地人脸库的标签
    labelFile.close()   
    data = np.loadtxt('faceData.txt',dtype=float)  
    
    # threshold of recognising photo
    threshold = 0.54
    # later save into label name
    name = input('Please input your name: ')
   
    font=cv2.FONT_HERSHEY_SIMPLEX           
    #框住人脸的矩形边框颜色       
    color = (0, 255, 0)
    
    #count pic, if test pics
    cnt_p = 0 
    
    #捕获指定摄像头的实时视频流
    cap = cv2.VideoCapture(0)#int(sys.argv[1]))
    
    #人脸识别分类器本地存储路径
    cascade_path = 'D:\\WM\\AI learning\\Ng - Deep Learning\\course4\\face recognistion\\haarcascade_frontalface_alt2.xml'    
    #D:\\WM\\AI learning\\Ng - Deep Learning\\course4\\face recognistion\\haarcascade_frontalface_alt2.xml


    while True:
        ret, frame = cap.read()   #读取一帧视频
        
        # kk is the return value of cv2.waitKey; the value is what user input
        kk = cv2.waitKey(1)

        #图像灰化，降低计算复杂度
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        #使用人脸识别分类器，读入分类器
        cascade = cv2.CascadeClassifier(cascade_path)                

        #利用分类器识别出哪个区域为人脸
        faceRects = cascade.detectMultiScale(frame_gray, scaleFactor = 1.2, minNeighbors = 3, minSize = (32, 32))        
        if len(faceRects) > 0:                 
            for faceRect in faceRects: 
                x, y, w, h = faceRect
                
                # if user input 's' ,then save pic
                if kk == ord('s'):
               
                    #截取脸部图像提交给模型识别这是谁
                    image = frame[y: y + h-5, x+10 : x + w -20]
                    image = resize_image(image)
                    embeddings = image_to_encoding(image, FRmodel) 
                #save image matrix and label 
                    data = np.concatenate((data, embeddings))                                                
                    label.append(name) 
                    
                    '''
                    # test photo size
                    path_make_dir = "imgs/img/"
                    cnt_p += 1
                    cv2.imwrite(path_make_dir + "/img_face_" + str(cnt_p) + ".jpg", image)
                    print("write：", str(path_make_dir) + "/img_face_" + str(cnt_p) + ".jpg")
                    '''
                                                    
                cv2.rectangle(frame, (x - 10, y - 10), (x + w +10, y + h +10), color, thickness = 2)
                cv2.putText(frame, "Faces: " + str(len(faceRects)), (20, 100), font, 0.8, (0, 255, 0), 1, cv2.LINE_AA)

                # 添加说明
                cv2.putText(frame, "Face Register", (20, 40), font, 1, (255, 255, 255), 1, cv2.LINE_AA)
                #cv2.putText(im_rd, "N: New face folder", (20, 350), font, 0.8, (255, 255, 255), 1, cv2.LINE_AA)
                cv2.putText(frame, "S: Save face", (20, 400), font, 0.8, (255, 255, 255), 1, cv2.LINE_AA)
                cv2.putText(frame, "Q: Quit", (20, 450), font, 0.8, (255, 255, 255), 1, cv2.LINE_AA)

    # 按下 'q' 键退出
        if kk & 0xFF== ord('q'):
            break    

                            
        cv2.imshow("Identify you", frame)
        '''        
        #等待10毫秒看是否有按键输入
        k = cv2.waitKey(10)
        #如果输入q则退出循环
        if k & 0xFF == ord('q'):
            break
        '''  
    np.savetxt('faceData.txt', data, fmt='%f')                                                          #保存人脸特征向量合成的矩阵到本地

    labelFile=open('label.txt','w')                                      
    json.dump(label, labelFile)                                                                         #使用json保存list到本地
    labelFile.close()
    #释放摄像头并销毁所有窗口
    cap.release()
    cv2.destroyAllWindows()


Please input your name: Travis_li


In [6]:
len(label), data.shape

(35, (35, 128))

In [7]:
label

['Aaron',
 'Aaron',
 'Bank19',
 'Bella',
 'dad',
 'dad',
 'dad',
 'Emma',
 'lisa',
 'lisa',
 'lisa',
 'lisa',
 'mum',
 'mum',
 'mum',
 'Robert',
 'xing18',
 'xing18',
 'xing18',
 'Yawei',
 'tiancai',
 'tiancai',
 'tiancai',
 'tiancai',
 'tiancai',
 'cen',
 'cen',
 'cen',
 'cen',
 'Travis_li',
 'Travis_li',
 'Travis_li',
 'Travis_li',
 'Travis_li',
 'Travis_li']

In [9]:
#-*- coding: utf-8 -*-
# recognize
if __name__ == '__main__':
    '''
    if len(sys.argv) != 2:
        print("Usage:%s camera_id\r\n" % (sys.argv[0]))
        sys.exit(0)
    '''   
    labelFile=open('label.txt','r')
    label = json.load(labelFile)                                                   #载入本地人脸库的标签
    labelFile.close()
    threshold = 0.54
    data = np.loadtxt('faceData.txt',dtype=float)  
    #model = Model()
    #model.load_model(file_path = './model/me.face.model.h5')    
               
    #框住人脸的矩形边框颜色       
    color = (0, 255, 0)
    
    #捕获指定摄像头的实时视频流
    cap = cv2.VideoCapture(0)#int(sys.argv[1]))
    
    #人脸识别分类器本地存储路径
    
    cascade_path = 'D:\\WM\\AI learning\\Ng - Deep Learning\\course4\\face recognistion\\haarcascade_frontalface_alt2.xml'
    #D:\\WM\\AI learning\\Ng - Deep Learning\\course4\\face recognistion\\haarcascade_frontalface_alt2.xml
    #循环检测识别人脸
    #while(cap.isOpened()):  # check !
    # capture frame-by-frame

    while True:
        ret, frame = cap.read()   #读取一帧视频
        
        #if ret: # check ! (some webcam's need a "warmup")
        # our operation on frame come here
        #图像灰化，降低计算复杂度
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        #使用人脸识别分类器，读入分类器
        #cascade = cv2.CascadeClassifier("D:\\WM\\AI learning\\Ng - Deep Learning\\course4\\face recognistion\\haarcascade_frontalface_alt2.xml")
        cascade = cv2.CascadeClassifier(cascade_path)                

        #利用分类器识别出哪个区域为人脸
        faceRects = cascade.detectMultiScale(frame_gray, scaleFactor = 1.2, minNeighbors = 3, minSize = (32, 32))        
        if len(faceRects) > 0:                 
            for faceRect in faceRects: 
                x, y, w, h = faceRect
                
                #截取脸部图像提交给模型识别这是谁
                image = frame[y - 10: y + h + 10, x - 10: x + w + 10]
                image = resize_image(image)
                #print('o: ',frame.shape)
                #print('now: ',image.shape)
                #_,identity = who_is_it(image, database, FRmodel)
                embeddings = image_to_encoding(image, FRmodel) 
                class_pre = findNearestClassForImage(embeddings, label)  
                
                
                #如果是“我”
                #if faceID == 0:                                                        
                cv2.rectangle(frame, (x - 10, y - 10), (x + w + 10, y + h + 10), color, thickness = 2)
                    
                #文字提示是谁
                cv2.putText(frame,class_pre, 
                            (x + 30, y + 30),                      #坐标
                            cv2.FONT_HERSHEY_SIMPLEX,              #字体
                            1,                                     #字号
                            (255,0,255),                           #颜色
                            2)                                     #字的线宽

                            
        cv2.imshow("Identify you", frame)
                  
        #等待10毫秒看是否有按键输入
        k = cv2.waitKey(10)
        #如果输入q则退出循环
        if k & 0xFF == ord('q'):
            break

    #释放摄像头并销毁所有窗口
    cap.release()
    cv2.destroyAllWindows()


In [ ]:
img = cv2.imread('imgs/Bank_19.jpg',1)#读取一张图片
#plt.imshow(img)
#img = resize_image(img)
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)#将图片转化成灰度

face_cascade = cv2.CascadeClassifier("D:\\WM\\AI learning\\Ng - Deep Learning\\course4\\face recognistion\\haarcascade_frontalface_alt2.xml")
#face_cascade.load('/haarcascade_frontalface_alt2.xml')#一定要告诉编译器文件所在的具体位置
#'''此文件是opencv的haar人脸特征分类器'''
faces = face_cascade.detectMultiScale(gray, 1.2,  minNeighbors =2, minSize = (32, 32))
print(faces)
image_set=[]
for (x,y,w,h) in faces:
    image = img[y - 10: y + h + 10, x - 10: x + w + 10]
    image_set.append(resize_image(image))
    #print(i)
    cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
    


cv2.namedWindow('img',0)#O表示显示窗口可以随意手动调节，1
cv2.imshow('img',img)
cv2.waitKey()
cv2.destroyAllWindows()

for i in range(len(image_set)):
    plt.imshow(image_set[i])


In [ ]:
for k,d in enumerate(faces):
    print(d[0])

In [ ]:
def recognition(img):
    dets = detector(img, 1)
    for k, d in enumerate(dets):
        
        print("Detection {}: Left: {} Top: {} Right: {} Bottom: {}".format(
            k, d.rect.left(), d.rect.top(), d.rect.right(), d.rect.bottom()))
        rec = dlib.rectangle(d.rect.left(),d.rect.top(),d.rect.right(),d.rect.bottom())
        print(rec.left(),rec.top(),rec.right(),rec.bottom())
        shape = sp(img, rec)
        face_descriptor = facerec.compute_face_descriptor(img, shape)        
        
        class_pre = findNearestClassForImage(face_descriptor, label)
        print(class_pre)
        cv2.rectangle(img, (rec.left(), rec.top()+10), (rec.right(), rec.bottom()), (0, 255, 0), 2)
        cv2.putText(img, class_pre , (rec.left(),rec.top()), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2, cv2.LINE_AA)

    cv2.imshow('image', img)

In [ ]:
cap = cv2.VideoCapture(0)

while(cap.isOpened()):  # check !
    # capture frame-by-frame
    ret, frame = cap.read()
    
    if ret: # check ! (some webcam's need a "warmup")
        # our operation on frame come here
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # Display the resulting frame
        cv2.imshow('frame', gray)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
# When everything is done release the capture
cap.release()
cv2.destroyAllWindows()

In [ ]:



# Dlib 预测器
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

name = input('what\'s your name? ')
# 创建 cv2 摄像头对象
cap = cv2.VideoCapture(0)

# cap.set(propId, value)
# 设置视频参数，propId 设置的视频参数，value 设置的参数值
cap.set(3, 480)

# 截图 screenshoot 的计数器
cnt_ss = 0

# 人脸截图的计数器
cnt_p = 0

# 存储人脸的文件夹
current_face_dir = 0

# 保存
path_make_dir = "imgs/img/"

path_csv = "data/csvs_from_camera/"

'''
# clear the old folders at first
def pre_clear():
    folders_rd = os.listdir(path_make_dir)
    for i in range(len(folders_rd)):
        shutil.rmtree(path_make_dir+folders_rd[i])

    csv_rd = os.listdir(path_csv)
    for i in range(len(csv_rd)):
        os.remove(path_csv+csv_rd[i])


# clear the exist folders of faces and csv
pre_clear()
'''

# 人脸种类数目的计数器
person_cnt = 0

# cap.isOpened（） 返回 true/false 检查初始化是否成功
while cap.isOpened():

    # cap.read()
    # 返回两个值：
    #    一个布尔值 true/false，用来判断读取视频是否成功/是否到视频末尾
    #    图像对象，图像的三维矩阵q
    flag, im_rd = cap.read()

    # 每帧数据延时 1ms，延时为 0 读取的是静态帧
    kk = cv2.waitKey(1)

    # 取灰度
    img_gray = cv2.cvtColor(im_rd, cv2.COLOR_RGB2GRAY)

    # 人脸数 rects
    rects = detector(img_gray, 0)

    # print(len(rects))q

    # 待会要写的字体
    font = cv2.FONT_HERSHEY_COMPLEX

    # 按下 'n' 新建存储人脸的文件夹
    if kk == ord('n'):
        person_cnt += 1
        # current_face_dir = path_make_dir + time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())
        current_face_dir = path_make_dir + "name_" + str(person_cnt)
        print('\n')
        for dirs in (os.listdir(path_make_dir)):
            if current_face_dir == path_make_dir + dirs:
                shutil.rmtree(current_face_dir)
                print("删除旧的文件夹:", current_face_dir)
        os.makedirs(current_face_dir)
        print("新建的人脸文件夹: ", current_face_dir)

        # 将人脸计数器清零
        cnt_p = 0

    if len(rects) != 0:
        # 检测到人脸

        # 矩形框
        for k, d in enumerate(rects):

            # 计算矩形大小
            # (x,y), (宽度width, 高度height)
            pos_start = tuple([d.left(), d.top()])
            pos_end = tuple([d.right(), d.bottom()])

            # 计算矩形框大小
            height = d.bottom() - d.top()
            width = d.right() - d.left()

            # 根据人脸大小生成空的图像
            cv2.rectangle(im_rd, tuple([d.left(), d.top()]), tuple([d.right(), d.bottom()]), (0, 255, 255), 2)
            im_blank = np.zeros((height, width, 3), np.uint8)

            # 按下 's' 保存摄像头中的人脸到本地
            if kk == ord('s'):
                cnt_p += 1
                for ii in range(height):
                    for jj in range(width):
                        im_blank[ii][jj] = im_rd[d.top() + ii][d.left() + jj]
                cv2.imwrite(current_face_dir + "/img_face_" + str(cnt_p) + ".jpg", im_blank)
                print("写入本地：", str(current_face_dir) + "/img_face_" + str(cnt_p) + ".jpg")

        # 显示人脸数
    cv2.putText(im_rd, "Faces: " + str(len(rects)), (20, 100), font, 0.8, (0, 255, 0), 1, cv2.LINE_AA)

    # 添加说明
    cv2.putText(im_rd, "Face Register", (20, 40), font, 1, (255, 255, 255), 1, cv2.LINE_AA)
    #cv2.putText(im_rd, "N: New face folder", (20, 350), font, 0.8, (255, 255, 255), 1, cv2.LINE_AA)
    cv2.putText(im_rd, "S: Save face", (20, 400), font, 0.8, (255, 255, 255), 1, cv2.LINE_AA)
    cv2.putText(im_rd, "Q: Quit", (20, 450), font, 0.8, (255, 255, 255), 1, cv2.LINE_AA)

    # 按下 'q' 键退出
    if kk == ord('q'):
        break

    # 窗口显示
    # cv2.namedWindow("camera", 0) # 如果需要摄像头窗口大小可调
    cv2.imshow("camera", im_rd)

# 释放摄像头
cap.release()

# 删除建立的窗口
cv2.destroyAllWindows()